# Wing Drag Minimization, with practical considerations

In our last example, we had a cautionary tale about using bad models and assumptions, and how you can easily find
yourself with nonsensical solutions if you throw together models without thinking about how they can be exploited.

Let's try doing another wing drag minimization problem, except this time let's model some important coupled effects,
such as:

* the mass of the wing, as well as how that scales with wing size and shape

* fuselage drag

* takeoff lift constraints

General problem structure is taken from Section 3 of "Geometric Programming for Aircraft Design Optimization" by W. Hoburg and P.
Abbeel. http://web.mit.edu/~whoburg/www/papers/hoburgabbeel2014.pdf

Several constants are tweaked in order to match the GPKit implementation available at: https://gpkit.readthedocs.io/en/latest/examples.html#simple-wing

In [1]:
import aerosandbox as asb
import aerosandbox.numpy as np

### Constants
form_factor = 1.2  # form factor [-]
oswalds_efficiency = 0.95  # Oswald efficiency factor [-]
viscosity = 1.78e-5  # viscosity of air [kg/m/s]
density = 1.23  # density of air [kg/m^3]
airfoil_thickness_fraction = 0.12  # airfoil thickness to chord ratio [-]
ultimate_load_factor = 3.8  # ultimate load factor [-]
airspeed_takeoff = 22  # takeoff speed [m/s]
CL_max = 1.5  # max CL with flaps down [-]
wetted_area_ratio = 2.05  # wetted area ratio [-]
W_W_coeff1 = 8.71e-5  # Wing Weight Coefficient 1 [1/m]
W_W_coeff2 = 45.24  # Wing Weight Coefficient 2 [Pa]
drag_area_fuselage = 0.031  # fuselage drag area [m^2]
weight_fuselage = 4940.0  # aircraft weight excluding wing [N]

opti = asb.Opti()  # initialize an optimization environment

### Variables
aspect_ratio = opti.variable(init_guess=10)  # aspect ratio
wing_area = opti.variable(init_guess=10)  # total wing area [m^2]
airspeed = opti.variable(init_guess=100)  # cruising speed [m/s]
weight = opti.variable(init_guess=10000)  # total aircraft weight [N]
CL = opti.variable(init_guess=1)  # Lift coefficient of wing [-]

### Models
# Aerodynamics model
CD_fuselage = drag_area_fuselage / wing_area
Re = (density / viscosity) * airspeed * (wing_area / aspect_ratio) ** 0.5
Cf = 0.074 / Re ** 0.2
CD_profile = form_factor * Cf * wetted_area_ratio
CD_induced = CL ** 2 / (np.pi * aspect_ratio * oswalds_efficiency)
CD = CD_fuselage + CD_profile + CD_induced
dynamic_pressure = 0.5 * density * airspeed ** 2
drag = dynamic_pressure * wing_area * CD
lift_cruise = dynamic_pressure * wing_area * CL
lift_takeoff = 0.5 * density * wing_area * CL_max * airspeed_takeoff ** 2

# Wing weight model
weight_wing_structural = W_W_coeff1 * (
        ultimate_load_factor * aspect_ratio ** 1.5 *
        (weight_fuselage * weight * wing_area) ** 0.5
) / airfoil_thickness_fraction
weight_wing_surface = W_W_coeff2 * wing_area
weight_wing = weight_wing_surface + weight_wing_structural

### Constraints
opti.subject_to([
    weight <= lift_cruise,
    weight <= lift_takeoff,
    weight == weight_fuselage + weight_wing
])

# Objective
opti.minimize(drag)

sol = opti.solve(max_iter=100)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       13

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.7311812e+03 5.54e+03 2.47e+01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

CasADi - 2024-04-05 19:41:32 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]


Now, we get a much more reasonable solution:

In [2]:
print(f"Minimum drag = {sol(drag)} N")
print(f"Aspect ratio = {sol(aspect_ratio)}")
print(f"Wing area = {sol(wing_area)} m^2")
print(f"Airspeed = {sol(airspeed)} m/s")
print(f"Weight = {sol(weight)} N")
print(f"C_L = {sol(CL)}")

Minimum drag = 303.07477258650505 N
Aspect ratio = 8.459983146202207
Wing area = 16.44179489082042 m^2
Airspeed = 38.15135783274743 m/s
Weight = 7341.0970007417345 N
C_L = 0.4987887182226508


We also see that we get a more reasonable L/D (albeit still high, as we are still neglecting many considerations, and the Schlichting turbulent flat plate model underestimates viscous losses here):

In [3]:
print(f"L/D = {sol(lift_cruise/drag)}")

L/D = 24.222065525595625


This illustrates just how important accurate modeling is when doing engineering design optimization - just like when
coding, an optimizer solves the problem that you actually give it, which is not necessarily the problem that you may
mean to solve.